# UserProxyAgent Introduction

## Overview

We covered `UserProxyAgent` in the previous section.  It is repeated here for your convenience.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
! pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

Note: Some agents require additional dependencies, which are included in the `autogen-ext` extras.

## `UserProxyAgent`

The `UserProxyAgent` is designed to simulate a human user. It takes user input and returns it as responses, often acting as the entry point for a conversation or a proxy for human feedback and intervention in multi-agent systems.

We haven't covered teams - this will be covered in a later tutorial.  For now just think of a team as multiple agents that work together until some termination criteria.

In [7]:
import asyncio
from autogen_agentchat.agents import UserProxyAgent, AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

async def run_user_proxy_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    
    # The assistant agent
    assistant = AssistantAgent(
        name="assistant",
        model_client=model_client,
        system_message="You are a helpful AI assistant."
    )
    
    # The user proxy agent
    user_proxy = UserProxyAgent(
        name="user_proxy",
    )
    
    print("\n🔄 Conversation with UserProxyAgent:")
    print("Enter TERMINATE to exit.")
    print("-" * 40)
    
    from autogen_agentchat.teams import RoundRobinGroupChat
    from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
    
    # Create a team with termination condition
    team = RoundRobinGroupChat(
        participants=[assistant, user_proxy],
        termination_condition=TextMentionTermination("TERMINATE") | MaxMessageTermination(max_messages=5)
    )
    
    # Run the team conversation
    await Console(team.run_stream(task="Tell me a joke."))    
    await model_client.close()

await run_user_proxy_example()


🔄 Conversation with UserProxyAgent:
----------------------------------------
---------- TextMessage (user) ----------
Tell me a joke.
---------- TextMessage (assistant) ----------
Why did the scarecrow win an award?

Because he was outstanding in his field!


Enter your response:  what do you call a donkey with three legs


---------- TextMessage (user_proxy) ----------
what do you call a donkey with three legs
---------- TextMessage (assistant) ----------
A wonky donkey!


Enter your response:  TERMINATE


---------- TextMessage (user_proxy) ----------
TERMINATE
